In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.preprocessing import Normalizer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score
from xgboost import XGBClassifier
import datetime
import matplotlib
from matplotlib import pyplot

In [2]:
import DataPrep
import error_evaluation

Using TensorFlow backend.


In [ ]:
X_Above_test, X_Above_train, y_Above_test, y_Above_train, X_Near_test, X_Near_train, y_Near_test, y_Near_train, X_Below_test, X_Below_train, y_Below_test, y_Below_train = DataPrep.prep_day1_danger_train_test('../Data/V1.1FeaturesWithLabels20131201To20180430.csv', ignore_extreme=True)

In [9]:
X_Above_test = pd.read_pickle('E:/Temp/X_test.pkl')
y_Above_test = pd.read_pickle('E:/Temp/y_test.pkl')

X_Above_train = np.load('E:/Temp/X_train_ADASYN.npy')
y_Above_train = np.load('E:/Temp/y_train_ADASYN.npy')

In [ ]:
#y_Above_train['o_Day1DangerAboveTreeline'].value_counts()

In [ ]:

#parameters derived from a several step parameter search
grid = {'min_child_weight': 5, 'n_estimators': 250, 'colsample_bytree': 0.9, 'reg_alpha': 1, 'gamma': 1, 'learning_rate': 0.1, 'subsample': 0.9, 'objective': 'multi:softmax', 'max_depth': 7}
#modelAbove = XGBClassifier(tree_method='gpu_hist', n_jobs=10)
modelAbove = XGBClassifier(n_jobs=10)
modelAbove.set_params(**grid)


In [ ]:
modelNear = XGBClassifier(n_jobs=10)
modelNear.set_params(**grid)

modelBelow = XGBClassifier(n_jobs=10)
modelBelow.set_params(**grid)

In [ ]:

%%time
yColumns = ['o_Day1DangerAboveTreeline']
val = modelAbove.fit(X_Above_train, y_Above_train)


In [ ]:
m = modelAboveGPU

In [ ]:
m.score(X_Above_test.values, y_Above_test['o_Day1DangerAboveTreeline'].values.ravel())


In [ ]:
#only look at critical points
y_Above_test_crit = y_Above_test[y_Above_test['IsCriticalPoint']==True]
X_Above_test_crit = X_Above_test.loc[y_Above_test_crit.index, :]
preds_crit = m.predict(X_Above_test_crit.values)

In [ ]:
#split in to forecast types
X_Above_test_cont = X_Above_test[X_Above_test['c_IsContenentalSnowpack']==True]
y_Above_test_cont = y_Above_test.loc[X_Above_test_cont.index, :]

X_Above_test_coast = X_Above_test[X_Above_test['c_IsCoastalSnowpack']==True]
y_Above_test_coast = y_Above_test.loc[X_Above_test_coast.index, :]

preds_cont = m.predict(X_Above_test_cont.values)
preds_coast = m.predict(X_Above_test_coast.values)

In [ ]:
preds = m.predict(X_Above_test.values)
y_Above_test['Prediction'] = preds

In [ ]:
#remove synthetic regions from the evaluation

synthetic_regions = ['Low West', 'Low East', 'In The Desert', 'In The Lake']
y_Above_test = y_Above_test[~y_Above_test['UnifiedRegion'].isin(synthetic_regions)]

In [ ]:
preds = y_Above_test['Prediction']

In [ ]:
import matplotlib.pyplot as plt
error_evaluation.evaluateSingleClassShort(y_Above_test['o_Day1DangerAboveTreeline'], preds)
cnf_matrix = confusion_matrix(y_Above_test['o_Day1DangerAboveTreeline'], preds)
plt.figure()
error_evaluation.plot_confusion_matrix(cnf_matrix, classes=['Low', 'Moderate', 'Considerable', 'High'], normalize=True,
                      title='Normalized confusion matrix')

plt.figure()
error_evaluation.plot_confusion_matrix(cnf_matrix, classes=['Low', 'Moderate', 'Considerable', 'High'],
                      title='Confusion matrix, without normalization')
plt.figure()
cnf_matrix_crit = confusion_matrix(y_Above_test_crit['o_Day1DangerAboveTreeline'], preds_crit)

error_evaluation.plot_confusion_matrix(cnf_matrix_crit, classes=['Low', 'Moderate', 'Considerable', 'High'], normalize=True,
                      title='Critcial Point Normalized confusion matrix')
plt.figure()
error_evaluation.plot_confusion_matrix(cnf_matrix_crit, classes=['Low', 'Moderate', 'Considerable', 'High'],
                      title='Critcial Point Confusion matrix, without normalization')

plt.figure()
error_evaluation.evaluateSingleClassShort(y_Above_test_cont['o_Day1DangerAboveTreeline'], preds_cont)
cnf_matrix_cont = confusion_matrix(y_Above_test_cont['o_Day1DangerAboveTreeline'], preds_cont)

error_evaluation.plot_confusion_matrix(cnf_matrix_cont, classes=['Low', 'Moderate', 'Considerable', 'High'], normalize=True,
                      title='Continental Forecast Normalized confusion matrix')
plt.figure()
error_evaluation.plot_confusion_matrix(cnf_matrix_cont, classes=['Low', 'Moderate', 'Considerable', 'High'],
                      title='Continental Forecast Confusion matrix, without normalization')

plt.figure()
error_evaluation.evaluateSingleClassShort(y_Above_test_coast['o_Day1DangerAboveTreeline'], preds_coast)
cnf_matrix_coast = confusion_matrix(y_Above_test_coast['o_Day1DangerAboveTreeline'], preds_coast)

error_evaluation.plot_confusion_matrix(cnf_matrix_coast, classes=['Low', 'Moderate', 'Considerable', 'High'], normalize=True,
                      title='Coastal Forecast Normalized confusion matrix')
plt.figure()
error_evaluation.plot_confusion_matrix(cnf_matrix_coast, classes=['Low', 'Moderate', 'Considerable', 'High'],
                      title='Coastal Forecast Confusion matrix, without normalization')
plt.show()

In [5]:
#'min_child_weight': 5, 'n_estimators': 500, 'colsample_bytree': 0.9, 'reg_alpha': 1, 'gamma': 1, 'learning_rate': 0.1, 'subsample': 0.9,'max_depth': 7, 'tree_method':'gpu_hist' 
grid = {'objective': 'multi:softmax', 'n_estimators': 1000, 'learning_rate': 0.01}
#grid = {'min_child_weight': 5, 'n_estimators': 250, 'colsample_bytree': 0.9, 'reg_alpha': 1, 'gamma': 1, 'learning_rate': 0.1, 'subsample': 0.9, 'objective': 'multi:softmax', 'max_depth': 7}
modelAboveGPU = XGBClassifier(n_jobs=10, tree_method='gpu_hist')
modelAboveGPU.set_params(**grid)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=10, nthread=None, objective='multi:softmax', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1, tree_method='gpu_hist')

In [6]:
#mapping = {'Low': 0, 'Moderate': 1, 'Considerable': 2, 'High': 3, 'Extreme': 4}
#df = y_Above_train.replace({'o_Day1DangerAboveTreeline': mapping})

In [7]:
%%time
val = modelAboveGPU.fit(X_Above_train, y_Above_train)

Wall time: 9min 43s


In [ ]:
valnear = modelNear.fit(X_Near_train, y_Near_train[yNearColumn].values.ravel())
valbelow = modelBelow.fit(X_Below_train, y_Below_train[yBelowColumn].values.ravel())

In [ ]:
resultAbove = modelAboveGPU.predict(X_Above_test.values)
#resultNear = modelNear.predict(X_Near_test)
#resultBelow = modelBelow.predict(X_Below_test)

In [ ]:
import matplotlib.pyplot as plt
error_evaluation.evaluateSingleClassShort(y_Above_test[yColumns[0]], resultAbove)
cnf_matrix = confusion_matrix(y_Above_test[yColumns[0]], resultAbove)
plt.figure()
error_evaluation.plot_confusion_matrix(cnf_matrix, classes=['Considerable', 'High', 'Low', 'Moderate'], normalize=True,
                      title='Normalized confusion matrix')

plt.figure()
error_evaluation.plot_confusion_matrix(cnf_matrix, classes=['Considerable', 'High', 'Low', 'Moderate'],
                      title='Confusion matrix, without normalization')
plt.show()

In [ ]:
evaluateSingleClassShort(y_Near_test['o_Day1DangerNearTreeline'], resultNear)

cnf_matrix = confusion_matrix(y_Near_test['o_Day1DangerNearTreeline'], resultNear)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Considerable', 'High', 'Low', 'Moderate'], normalize=True,
                      title='Normalized confusion matrix')
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Considerable', 'High', 'Low', 'Moderate'],
                      title='Confusion matrix, without normalization')
plt.show()

In [ ]:
evaluateSingleClassShort(y_Below_test['o_Day1DangerBelowTreeline'], resultBelow)
cnf_matrix = confusion_matrix(y_Below_test['o_Day1DangerBelowTreeline'], resultBelow)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Considerable', 'High', 'Low', 'Moderate'], normalize=True,
                      title='Normalized confusion matrix')
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Considerable', 'High', 'Low', 'Moderate'],
                      title='Confusion matrix, without normalization')
plt.show()

In [ ]:
#check out the feature importances
featureImportance = dict(zip(X_Above_train.columns, modelAbove.feature_importances_))
sortedFeatureImportance = sorted(featureImportance, key=featureImportance.get, reverse=True)

In [ ]:
sortedFeatureImportance[:100]

In [ ]:
#save the models
modelAbove._Booster.save_model("ModelAboveV1.bin")
modelNear._Booster.save_model("ModelNearV1.bin")
modelBelow._Booster.save_model("ModelBelowV1.bin")

In [ ]:
%%time #3 mins
# grid search
model = XGBClassifier(tree_method='gpu_hist')
learning_rate = [0.01]
n_estimators = [100, 200]
#max_depth = range(3,10,2)
#min_child_weight = range(1,6,2)
#gamma = [i/10.0 for i in range(0,5)]
param_grid = dict(learning_rate=learning_rate, n_estimators=n_estimators, objective=["gpu:multi:softmax"])
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=8, cv=kfold)
#(X_Near_train, y_Near_train[yNearColumn].values.ravel()
grid_result = grid_search.fit(X_Above_train.values, y_Above_train[yColumns].values.ravel())
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
%%time #1 hour 30 mins
# grid search
model = XGBClassifier()
learning_rate = [0.01]
n_estimators = [100, 200]
#max_depth = range(3,10,2)
#min_child_weight = range(1,6,2)
#gamma = [i/10.0 for i in range(0,5)]
param_grid = dict(learning_rate=learning_rate, n_estimators=n_estimators, objective=["multi:softmax"])
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=8, cv=kfold)
#(X_Near_train, y_Near_train[yNearColumn].values.ravel()
grid_result = grid_search.fit(X_Above_train.values, y_Above_train[yColumns].values.ravel())
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
%%time
# grid search
model = XGBClassifier(tree_method='gpu_hist')
learning_rate = [.1]
n_estimators = [400, 700, 900]
max_depth = range(3,10,2)
min_child_weight = range(1,6,2)
gamma = [i/10.0 for i in range(0,5)]
param_grid = dict(learning_rate=learning_rate, n_estimators=n_estimators,max_depth=max_depth, min_child_weight=min_child_weight, gamma=gamma, objective=["gpu:multi:softmax"])
kfold = StratifiedKFold(n_splits=7, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", cv=kfold, verbose=20)
#(X_Near_train, y_Near_train[yNearColumn].values.ravel()
grid_result = grid_search.fit(X_Above_train.values, y_Above_train[yColumns].values.ravel())
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))